In [49]:
import pandas as pd
import numpy as np

user = 'DiciDicee'

df = pd.read_csv(f"~/.chess-stats/game-synthesis/{user}/games.tsv", sep='\t')

pd.set_option('display.min_rows', 50)
pd.set_option('display.max_rows', 100)

In [38]:
def normalized_chess_com_openings():
    openings = pd.read_csv('/Users/davidcourtinot/repos/personal/chess-stats/chess-com-openings.tsv', sep='\t')
    openings['source'] = 'chess.com scraping'
    openings['trustworthiness'] = 3
    openings['eco'] = None
    return openings

def normalized_kaggle_openings():
    openings = pd.read_csv('/Users/davidcourtinot/repos/personal/chess-stats/kaggle-openings.csv', sep=',')
    openings = openings.drop(columns=[
        '#', 'Num Games', 'Opponent Win %', 'Draw %', 'Player Win %', 'Colour', 'Avg Player', 'Num Games', 
        'Last Played', 'Perf Rating', 'moves_list', 'move1w', 'move1b', 'move2w', 'move2b', 'move3w', 'move3b', 
        'move4w', 'move4b', 'White_win%', 'Black_win%', 'White_odds', 'White_Wins', 'Black_Wins'
    ])
    openings = openings.rename(columns={
        'Opening': 'opening_name', 
        "ECO": "eco",
        'Moves': 'opening_moves'
    })
    openings['opening_moves'] = openings['opening_moves'].str.replace('1.00E+0', '1.e')
    openings['source'] = 'Kaggle'
    openings['trustworthiness'] = 2
    return openings    

def normalized_chess_canvas_openings():
    openings = pd.read_csv('/Users/davidcourtinot/repos/personal/chess-stats/chess-canvas-openings.csv', sep=',')
    openings = openings.rename(columns={
        'name': 'opening_name', 
        "ECO": "eco",
        'moves': 'opening_moves'
    })
    openings['opening_moves'] = openings['opening_moves'].str.replace('1.00E+0', '1.e')
    openings['opening_moves'] = openings['opening_moves'].str.strip()
    openings['source'] = 'Github chess-canvas'
    openings['trustworthiness'] = 1
    return openings        

def reconcile_openings(openings_dfs):
    openings_all_sources = pd.concat(openings_dfs)
    openings_all_sources_sorted = openings_all_sources.sort_values(by=['opening_moves', 'trustworthiness'])
    return openings_all_sources_sorted.groupby('opening_moves').last().reset_index()

chess_com_openings = normalized_chess_com_openings()
kaggle_openings = normalized_kaggle_openings()
chess_canvas_openings = normalized_chess_canvas_openings()

reconciled_openings = reconcile_openings([chess_com_openings, kaggle_openings, chess_canvas_openings])

In [81]:
def load_games(user):
    games = pd.read_csv(f"~/.chess-stats/game-synthesis/{user}/games.tsv", sep='\t')
    games = games.drop(columns=['variant'])
    games = games[games['eco'] != '?']
    games = games.rename(columns={'moves': 'game_moves'})
    games['game_moves'] = games['game_moves'].str.replace(r'\s\{.*?\}', '', regex=True)
    games['game_moves'] = games['game_moves'].str.replace(r'\d+\.{3}\s', '', regex=True)
    games['game_moves'] = games['game_moves'].str.replace(r'\.\s', '.', regex=True)
    return games

games = load_games('DiciDicee')

In [83]:
def expand_to_prefix_moves(df, max_prefixes):
    df['game_moves_list'] = df['game_moves'].str.split()

    def get_prefixes(moves):
        prefixes = []
        current_prefix = []
        for move in moves:
            current_prefix.append(move)
            prefixes.append(' '.join(current_prefix))
            if len(prefixes) == max_prefixes:
                break
        return prefixes

    df['game_moves_prefix'] = df['game_moves_list'].apply(get_prefixes)
    df_expanded = df.explode('game_moves_prefix')
    df_expanded = df_expanded.drop(columns=['game_moves_list'])
    
    # will be used to take the longest common prefix, which means it is the most precise characterization of the opening
    df_expanded['game_moves_prefix_length'] = df_expanded['game_moves_prefix'].str.len()

    return df_expanded

expanded_games = expand_to_prefix_moves(games.copy(), 30)

In [92]:
def enrich_games_with_opening_metadata(expanded_games, openings):
    df = expanded_games.merge(openings, left_on='game_moves_prefix', right_on='opening_moves', how='inner') 
    df['eco'] = df['eco_x'].combine_first(df['eco_y'])
    df = df.sort_values(by=['link', 'game_moves_prefix_length'])
    df = df.groupby('link').last().reset_index() # keep only the most specific match (longest opening line)
    df = df.drop(columns=['trustworthiness', 'eco_x', 'eco_y', 'game_moves_prefix', 'game_moves_prefix_length'])
    return df

games_with_openings = enrich_games_with_opening_metadata(expanded_games, reconciled_openings)

In [95]:
opening_outcomes = games[df["date"] >= "2024-01"][["date", "color", "result", "opening_family", "time_control"]]
opening_outcomes = opening_outcomes[opening_outcomes['time_control'] != 'Bullet']

/var/folders/h5/jw2kqsmn6db13xxc66616ktc0000gp/T/ipykernel_76884/1533561437.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  opening_outcomes = games[df["date"] >= "2024-01"][["date", "color", "result", "opening_family", "time_control"]]


In [93]:
opening_outcomes_count = opening_outcomes.groupby(['color', 'opening_family', 'result']).size().reset_index(name='count')
opening_family_totals = opening_outcomes_count.groupby(['color', 'opening_family'])['count'].transform('sum')

opening_outcomes_count['percentage'] = (opening_outcomes_count['count'] / opening_family_totals) * 100
opening_outcomes_count['games_count'] = opening_family_totals

def convert_score_to_result(df, score_col = 'result', color_col = 'color'):
    conditions = [
        # 1. Win: (White and 1-0) OR (Black and 0-1)
        ((df[color_col] == 'White') & (df[score_col] == '1-0')) | 
        ((df[color_col] == 'Black') & (df[score_col] == '0-1')),
        
        # 2. Loss: (White and 0-1) OR (Black and 1-0)
        ((df[color_col] == 'White') & (df[score_col] == '0-1')) | 
        ((df[color_col] == 'Black') & (df[score_col] == '1-0')),
        
        # 3. Draw: 1/2-1/2 for either color
        (df[score_col] == '1/2-1/2')
    ]
    
    choices = ['Win', 'Loss', 'Draw']
    
    df['result'] = np.select(conditions, choices, default='Unknown')
    
    return df


opening_outcomes_count = convert_score_to_result(opening_outcomes_count)
opening_outcomes_count = opening_outcomes_count.sort_values(by=['games_count', 'opening_family', 'result'], ascending=False)

In [94]:
def identify_weak_openings(df):
    win_rate_map = df[df['result'] == 'Win'].set_index(['color', 'opening_family'])['percentage']
    df['opening_win_rate'] = df.set_index(['color', 'opening_family']).index.map(win_rate_map)
    
    loss_rate_map = df[df['result'] == 'Loss'].set_index(['color', 'opening_family'])['percentage']
    df['opening_loss_rate'] = df.set_index(['color', 'opening_family']).index.map(loss_rate_map)
    
    df_filtered = (
        df[df['opening_win_rate'] < df['opening_loss_rate']]
            .drop(columns=['opening_win_rate', 'opening_loss_rate'])
    )
    return df_filtered

weak_openings = identify_weak_openings(opening_outcomes_count)